In [96]:
from __future__ import print_function

import pandas as pd
import numpy as np

import os
import codecs
import pickle
import json

import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.externals import joblib
from sklearn.manifold import MDS

from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.stem.snowball import SnowballStemmer
import re
import mpld3

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [2]:
from sentiment_cluster import Sentiment_Cluster

In [3]:
san_francisco = Sentiment_Cluster('data/san-francisco/')

In [4]:
san_francisco._get_file_names()

In [5]:
len(san_francisco.file_names)

214

In [6]:
# randomly choose a hotel
hotel = random.choice(san_francisco.file_names)
hotel

'usa_san francisco_beresford_arms'

In [7]:
# get all hotel reviews for random hotel
san_francisco._get_hotel_reviews(hotel)

In [8]:
partial_review = san_francisco.reviews[0][:100]
partial_review

'We stayed here in November in the early stages of a post-retirement backpacking trip around the worl'

In [9]:
# test out sentiment analysis
analyzer = SentimentIntensityAnalyzer()

In [10]:
analyzer.polarity_scores(partial_review)

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [11]:
analyzer.polarity_scores(partial_review)

{'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}

In [12]:
analyzer.polarity_scores(san_francisco.reviews[0])

{'compound': 0.9924, 'neg': 0.016, 'neu': 0.858, 'pos': 0.126}

In [13]:
# test clustering sentiments

positives = []
negatives = []
neutrals = []

In [14]:
def cluter_review_sentiments(reviews):
        
        ''' splits all reviews into Positive, Negative, or Neutral clusters '''
        
        sent_analyzer = SentimentIntensityAnalyzer()
        
        # loop through all reviews and apply analyzer
        for review in reviews:
            if sent_analyzer.polarity_scores(review)['compound'] >= .4:
                positives.append(review)
            elif sent_analyzer.polarity_scores(review)['compound'] <= -.4:
                negatives.append(review)
            else:
                neutrals.append(review)
                
        

In [15]:
def cluter_review_bigrams_sentiments(reviews):
        
        ''' splits all reviews into Positive, Negative, or Neutral clusters '''
        
        sent_analyzer = SentimentIntensityAnalyzer()
        
        # loop through all reviews and apply analyzer
        for review in reviews:
            if sent_analyzer.polarity_scores(review)['compound'] >= .4:
                positives.append(list(nltk.bigrams(review.split())))
            elif sent_analyzer.polarity_scores(review)['compound'] <= -.4:
                negatives.append(list(nltk.bigrams(review.split())))
            else:
                neutrals.append(list(nltk.bigrams(review.split())))
                
        

In [16]:
# lst = ['This hotel is HORRIBLE!', 'i can deal with it', 'I LOVE THIS', 'ive been better', 
#        'such crap, never coming again', 'so-so']

In [17]:
cluter_review_sentiments(san_francisco.reviews)

In [18]:
# cluter_review_bigrams_sentiments(san_francisco.reviews)

In [19]:
len(positives), len(negatives), len(neutrals)

(141, 9, 21)

In [20]:
# negatives

In [21]:
negatives[0]

'I booked this hotel because of all the favourite reviews. Very disappointed after checking into the room. There were quite a few homeless people roaming on the block. Had the junior suite (really was just 2 queen beds plus a small kitechen area). The kitchen appliances were so dirty and rusty that I dared not touch them (let alone using them). What was killing me was the noise level from the street. I was on the 4th floor but make no mistake, it is just the same as street level. The windows provided no sound insulation at all. Right next to the busy Post St, you could hear all sorts of noises in the wee morning hours (pedestrians talking, car doors banging, bellboy unloading suitcases, all the speeding cars etc). Even with ear plugs, I had a hard time falling asleep. Had to shortern our stay in that hotel and found a much better hotel on Howard street. Again, I do not know why this hotel is receiving all these good reviews. I bet that all the rooms facing the busy streets have the sam

In [22]:
analyzer.polarity_scores(negatives[0])

{'compound': -0.9015, 'neg': 0.127, 'neu': 0.834, 'pos': 0.039}

In [23]:
# positives

In [24]:
analyzer.polarity_scores(positives[0])

{'compound': 0.9924, 'neg': 0.016, 'neu': 0.858, 'pos': 0.126}

In [25]:
analyzer.polarity_scores('"This hotel is HORRIBLE! I can deal with old but, it was dirty')

{'compound': -0.7756, 'neg': 0.403, 'neu': 0.597, 'pos': 0.0}

In [87]:
# # # test tfidf on positives

# tfidf_vectorizer = TfidfVectorizer(max_df=0.6,
#                                  min_df=0.2, stop_words='english',
#                                  use_idf=True, ngram_range=(2, 4))

# # tfidf_vectorizer = TfidfVectorizer(stop_words='english',
# #                                  use_idf=True, ngram_range=(1, 3))

# positives_tfidf_matrix = tfidf_vectorizer.fit_transform(positives)

# print("tfidf shape: {}".format(positives_tfidf_matrix.shape))

# # test tfidf on negatives

# neg_tfidf_vectorizer = TfidfVectorizer(max_df=0.6,
#                                  min_df=0.2,
#                                  stop_words='english',
#                                  use_idf=True, ngram_range=(2, 4))

# negatives_tfidf_matrix = neg_tfidf_vectorizer.fit_transform(negatives)

# # print("tfidf shape: {}".format(negatives_tfidf_matrix.shape))

tfidf shape: (141, 5)


In [144]:
# test countvectorizer

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                ngram_range=(2, 2),
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(positives)
print(dtm_tf.shape)

(141, 31)


In [146]:
# test countvectorizer for neg

neg_tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                ngram_range=(2, 2),
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 1, 
                                min_df = 1)
neg_dtm_tf = neg_tf_vectorizer.fit_transform(negatives)
print(neg_dtm_tf.shape)

(9, 901)


In [130]:
# test LDA

# lda_tfidf = LatentDirichletAllocation(n_topics=5)

In [131]:
# neg_lda_tfidf = LatentDirichletAllocation(n_topics=5)

In [149]:
# for TF
lda_tf = LatentDirichletAllocation(n_topics=5)
lda_tf.fit(dtm_tf)

/Users/steventa/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/steventa/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=5,
             perp_tol=0.1, random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [150]:
# for TF for neg
neg_lda_tf = LatentDirichletAllocation(n_topics=5)
neg_lda_tf.fit(neg_dtm_tf)

/Users/steventa/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)
/Users/steventa/anaconda3/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:536: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=5,
             perp_tol=0.1, random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [151]:
# lda_tfidf.fit(positives_tfidf_matrix)

In [152]:
# neg_lda_tfidf.fit(negatives_tfidf_matrix)

#### We have ratings for 5 topics. Can our topic modeling find it?

- CLEANLINESS	
- ROOM	
- SERVICE	
- LOCATION
- VALUE	

In [153]:
# visual LDAvis

# vis_data = pyLDAvis.sklearn.prepare(lda_tfidf, positives_tfidf_matrix, tfidf_vectorizer)

In [154]:
# pyLDAvis.display(vis_data)

In [155]:
# visual LDAvis

# neg_vis_data = pyLDAvis.sklearn.prepare(neg_lda_tfidf, negatives_tfidf_matrix, neg_tfidf_vectorizer)

In [156]:
# pyLDAvis.display(neg_vis_data)

In [157]:
# visualiza CV

pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      35.889474        1       1  0.254497 -0.029460
1      24.981097        1       2  0.029380 -0.022861
3      14.912201        1       3 -0.010900 -0.129507
2      12.599210        1       4 -0.261615 -0.035258
4      11.618018        1       5 -0.011362  0.217086, topic_info=     Category       Freq                   Term      Total  loglift  logprob
term                                                                        
2     Default  58.000000         beresford arms  58.000000  30.0000  30.0000
27    Default  18.000000           street noise  18.000000  29.0000  29.0000
5     Default  18.000000            close union  18.000000  28.0000  28.0000
19    Default  11.000000             room clean  11.000000  27.0000  27.0000
26    Default  18.000000          stayed nights  18.000000  26.0000  26.0000
13    Default  15.000000         large bathroom  15.000000  25.0000  25.0000
20    Default  22.000000            rooms clean  22.000000  24.0000  24.0000
23    Default  17.000000         staff friendly  17.000000  23.0000  23.0000
21    Default  10.000000               san fran  10.000000  22.0000  22.0000
25    Default  24.000000       stayed beresford  24.000000  21.0000  21.0000
12    Default  13.000000           junior suite  13.000000  20.0000  20.0000
10    Default  12.000000             good value  12.000000  19.0000  19.0000
4     Default  17.000000              cable car  17.000000  18.0000  18.0000
8     Default  17.000000          free internet  17.000000  17.0000  17.0000
3     Default  13.000000           blocks union  13.000000  16.0000  16.0000
9     Default  20.000000       friendly helpful  20.000000  15.0000  15.0000
24    Default  13.000000          staff helpful  13.000000  14.0000  14.0000
22    Default  37.000000          san francisco  37.000000  13.0000  13.0000
18    Default  10.000000        recommend hotel  10.000000  12.0000  12.0000
30    Default  30.000000            wine cheese  30.000000  11.0000  11.0000
1     Default  14.000000             arms hotel  14.000000  10.0000  10.0000
11    Default   8.000000        internet access   8.000000   9.0000   9.0000
6     Default   7.000000             coffee tea   7.000000   8.0000   8.0000
16    Default  10.000000            older hotel  10.000000   7.0000   7.0000
14    Default   6.000000             near union   6.000000   6.0000   6.0000
0     Default  14.000000         afternoon wine  14.000000   5.0000   5.0000
7     Default  24.000000  continental breakfast  24.000000   4.0000   4.0000
17    Default  10.000000             queen beds  10.000000   3.0000   3.0000
29    Default  15.000000       walking distance  15.000000   2.0000   2.0000
15    Default   9.000000             nice touch   9.000000   1.0000   1.0000
...       ...        ...                    ...        ...      ...      ...
27     Topic5  12.139055           street noise  18.661463   1.7226  -1.6295
5      Topic5  10.994390            close union  18.447865   1.6350  -1.7285
20     Topic5   8.783630            rooms clean  22.469800   1.2133  -1.9530
14     Topic5   2.365852             near union   6.561957   1.1325  -3.2648
6      Topic5   1.682685             coffee tea   7.649001   0.6384  -3.6055
16     Topic5   1.904676            older hotel  10.912611   0.4070  -3.4816
4      Topic5   2.825516              cable car  17.629328   0.3217  -3.0872
30     Topic5   3.159971            wine cheese  30.741774  -0.1224  -2.9753
7      Topic5   1.645605  continental breakfast  24.040894  -0.5290  -3.6278
28     Topic5   0.340828             walk union   5.980939  -0.7123  -5.2023
11     Topic5   0.418809        internet access   8.541989  -0.8627  -4.9962
15     Topic5   0.393525             nice touch   9.543424  -1.0359  -5.0585
8      Topic5   0.682045          free internet  17.373080  -1.0850  -4.5086
18     Topic5   0.388743        recom

In [158]:
# visualiza CV for neg

pyLDAvis.sklearn.prepare(neg_lda_tf, neg_dtm_tf, neg_tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      27.590136        1       1 -0.001335 -0.057743
0      27.481146        1       2 -0.050295  0.030395
4      25.722332        1       3  0.051728  0.027980
1      10.208154        1       4 -0.002100 -0.001361
2       8.998232        1       5  0.002002  0.000728, topic_info=     Category      Freq                 Term     Total  loglift  logprob
term                                                                    
633   Default  1.000000    reasonably priced  1.000000  30.0000  30.0000
440   Default  0.000000    linoleum bathroom  0.000000  29.0000  29.0000
564   Default  0.000000           penny dump  0.000000  28.0000  28.0000
562   Default  0.000000            pay penny  0.000000  27.0000  27.0000
242   Default  0.000000            dump work  0.000000  26.0000  26.0000
805   Default  0.000000           told today  0.000000  25.0000  25.0000
532   Default  0.000000          old cracked  0.000000  24.0000  24.0000
1     Default  0.000000  absolutely refusing  0.000000  23.0000  23.0000
773   Default  0.000000            suite got  0.000000  22.0000  22.0000
90    Default  0.000000      booked internet  0.000000  21.0000  21.0000
360   Default  0.000000     hotel absolutely  0.000000  20.0000  20.0000
889   Default  0.000000          work number  0.000000  19.0000  19.0000
468   Default  0.000000     luggage insisted  0.000000  18.0000  18.0000
64    Default  0.000000        bedroom filty  0.000000  17.0000  17.0000
216   Default  0.000000      different place  0.000000  16.0000  16.0000
675   Default  0.000000        san francisco  0.000000  15.0000  15.0000
891   Default  0.000000       worst horrible  0.000000  14.0000  14.0000
104   Default  0.000000         burns carpet  0.000000  13.0000  13.0000
395   Default  0.000000       internet rooms  0.000000  12.0000  12.0000
603   Default  0.000000         price points  0.000000  11.0000  11.0000
749   Default  0.000000          stay matter  0.000000  10.0000  10.0000
352   Default  0.000000          holes burns  0.000000   9.0000   9.0000
117   Default  0.000000            care time  0.000000   8.0000   8.0000
91    Default  0.000000        booked junior  0.000000   7.0000   7.0000
24    Default  0.000000           area hotel  0.000000   6.0000   6.0000
434   Default  0.000000            life left  0.000000   5.0000   5.0000
604   Default  0.000000          price worth  0.000000   4.0000   4.0000
23    Default  0.000000           area gross  0.000000   3.0000   3.0000
771   Default  0.000000       suggested desk  0.000000   2.0000   2.0000
570   Default  0.000000      personal travel  0.000000   1.0000   1.0000
...       ...       ...                  ...       ...      ...      ...
891    Topic5  0.240619       worst horrible  0.821369   1.1804  -5.8287
360    Topic5  0.243456     hotel absolutely  0.831301   1.1801  -5.8170
803    Topic5  0.236211           today area  0.807236   1.1793  -5.8472
24     Topic5  0.239452           area hotel  0.820842   1.1762  -5.8336
117    Topic5  0.240244            care time  0.824487   1.1750  -5.8303
562    Topic5  0.245808            pay penny  0.844452   1.1740  -5.8074
91     Topic5  0.239038        booked junior  0.821626   1.1735  -5.8353
773    Topic5  0.243867            suite got  0.838982   1.1726  -5.8153
188    Topic5  0.233901            dark stay  0.807243   1.1694  -5.8571
352    Topic5  0.240023          holes burns  0.828945   1.1687  -5.8312
434    Topic5  0.239855            life left  0.829209   1.1677  -5.8319
1      Topic5  0.244843  absolutely refusing  0.846580   1.1676  -5.8113
604    Topic5  0.239390          price worth  0.827774   1.1675  -5.8339
751    Topic5  0.235904        stayed booked  0.816021   1.1671  -5.8485
570    Topic5  0.237624      personal travel  0.822097   1.1670  -5.8413
216    Topic5  0.241969      different place  0.838242   1.1656  -5.8231
64